In [3]:
import numpy as np
import pandas as pd
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from konlpy.tag import Okt

In [4]:
data = pd.read_csv('./test_review.csv')
data.columns = ['star', 'date', 'store', 'review', 'help']

In [5]:
from hanspell import spell_checker
import re

def data_preprocessing(review):
    pattern = '[^\w\s]'         # 특수기호제거'[^\w\sㄱ-ㅎㅏ-ㅣ]' 
    review = re.sub(pattern=pattern, repl='', string=review)
    review = re.sub(pattern='[\s]+', repl=' ', string=review)
    return spell_checker.check(review).checked

In [6]:
from tqdm import tqdm
tqdm.pandas()

data['review'] = data['review'].progress_apply(data_preprocessing)

100%|██████████| 29/29 [00:02<00:00, 11.00it/s]


In [7]:
data

,star,date,store,review,help
0,5,2022.08.08,쿠팡(주),,6
1,4,2022.08.04,쿠팡(주),,5
2,5,2022.08.03,쿠팡(주),,5
3,5,2022.08.01,쿠팡(주),,1
4,5,2022.07.30,쿠팡(주),,13
5,5,2022.08.08,쿠팡(주),,1
6,5,2022.07.27,쿠팡(주),좋아요 아이들이 방학 동안 게임하는데 TV로 연결해서 했거든요 그런데 쿠팡 체험단이...,5
7,5,2022.08.02,쿠팡(주),,1
8,4,2022.07.27,쿠팡(주),,2
9,5,2022.09.11,쿠팡(주),굿굿 사무용으로 듀얼 모니터가 필요한데 인터넷용으로 사용할 거라 고사양이나 비싼 건...,0


In [8]:
test_data = []
for idx, d in enumerate(data['review']):
    test_data.append(d)
    
test_data = ' '.join(test_data)

In [9]:
data['review'].tolist()

['',
 '',
 '',
 '',
 '',
 '',
 '좋아요 아이들이 방학 동안 게임하는데 TV로 연결해서 했거든요 그런데 쿠팡 체험단이 되어서 모니터 보자마자 클릭클릭 무료로 제공받았습니다 설치는 정말 간단해요 나사 4개만 끼우면 끝 뒤에 전원 코드 꽂고 게임기 연결하니 바로 되네요 ᄒ오오 정말 좋아요 사이즈도 정말 적당하고 화질이야 워낙 요즘 TV들이 고화질이라 따라갈 순 없지만 이거 아주 좋아요 그냥 식탁 한쪽에 놓고 사용하기 너무 좋아요 딱하나 아쉬운 건 스피커 그래서 조만간 연결 잭 사서 스피커에 연결해서 사용해야 할 것 같아요 스포츠 게임하려면 이어폰 끼고 할 수 없으니까요 저희는 컴퓨터나 노트북 연결해서 쓸 게 아니라서 스피커가 꼭 필요하거든요 그것만 연결하면 200프로 만족할 거 같아요 가볍고 화질도 좋고 오른쪽 뒷면에 전원 등 조절하는 게 있어요 너무 만족합니다 블루투스 연결이 되네요ㅎㅎ 집에 있는 스피커랑 블루투스로 연결해서 이제 스피커도 해결돼서 게임 중입니다 쿠팡 체험단 이벤트로 상품을 무료로 제공받아 작성한 구매 후기입니다',
 '',
 '',
 '굿굿 사무용으로 듀얼 모니터가 필요한데 인터넷용으로 사용할 거라 고사양이나 비싼 건 필요 없이 보조용으로 쓸 모니터를 찾던 중 금액도 저렴해서 구매했습니다 추석 연휴 일요일인데 로켓 와우로 다음날 새벽에 받아볼 수 있다니 ㅜㅜ 너무 만족스러워요 모니터를 사게 된 이유 컴맹이라 컴퓨터로는 인터넷만 사용 사무용 보조 모니터가 필요 조립은 너무 간단했고 동봉된 HDMI 선만 연결하니 듀얼 모니터 완성 듀얼 모니터가 생겼으니 이제 열이 해봐야겠어요',
 '이전부터 한성 22인치로 사무용 쓰고 있었는데 좀 불편한 감이 있었지만 저렴한 맛으로 썼었습니다 근데 업그레이드됐네요 가격은 그대로인데 너무 좋아져서 오히려 당황했습니다 항상 좋은 방향으로 발전하시는 거 같아 추천합니다',
 '구성비 재택근무 시 적당히 쓰려고 구매했어요 사이즈는 생각보다 작은 편이라 한 사이즈 크게 살걸 살짝 후회됩니다 ㅠㅠ 설

In [28]:
test_data = '사무용으로  딱! 현관문 앞에 배송되어 왔을 때 첨엔 깜짝 놀랬어요. 겉박스가  찌그러지고 테이핑 처리 된것이 다 떨어져 안에 내용물이 다 보였거든요.  하지만 다행인것이 겉박스 안에 모니터 메인박스가 또 있어서 제품 손상이 안 되었지요. 걱정스럽게 뜯어 봤는데 모니터가 손상되지 않도록 스티로폼이 모서리에 덪되어 안전하게 포장이 되어 있었어요. 겉포장 박스만 찌그러지고 터진거.. 이번꺼는 배송해주신 택배기사님이 조심스럽게 취급을 안하신듯 싶어요. ㅜㅜ  내용물을 꺼내 부속품들을 확인하고 조립을 해보았는데 어렵지 않았습니다. 제품의 구성과  조립방법은 다른분들의 리뷰에 상세히 나와 있으니 생략! ㅋ  다른분들의 리뷰를 보면 거의 컴퓨터본체와 연결해서 테스트를 많이 하셨길래 전 동영상에 올린것처럼 노트북과 연결을 해보았어요. 노트북과 함께 사용할때도 편리하며 잘 되었습니다.  주로 문서 작업용으로 사용할거라서 그것에 맞추어 테스트를 해보았습니다. 자유롭게 사용자에 맞추어 모니터 각도 조절도 되니 목의 피로도에 도움이 될듯 싶네요 사용 했을때 자료를 모니터에 띄어놓고 메인 모니터로 문서작업을 하니 무척 편하더라구요. 모니터 한개로 창을 두개 띄워서 작업할때보다 일의 효율성도 늘고 자료 찾기도 더 쉬웠습니다.  컴퓨터 게임은 안하는 편이지만 그래픽카드 출력하는 프레임과 모니터의  재생빈도가 엇갈리며  발생하는 찢어짐  현상을 없애주는 Freesyne 호환기술을 지원하여 화면  왜곡없이 빠르고 매끄러운 게임플레이가 가능하다고 하다네요. 하지만 전 문서작업용 용도.ㅋ  논글레어 눈부심방지 코팅과 플리커플리, 블루라이트 제어기술  탑재로 화면 깜박임이 없어 사용자를 보호 한다고 하던데  문서 작업하면서 눈의 피로도에도 도움이 될것 같습니다. 일부러 눈의 시력이 좋더라도 눈을 보호하기 위하여 블루라이트가 들어있는 시력보호안경을 끼기도 하니까...  디자인도 심플하니 깔끔해보이고 가성비도 이정도면... 용도에 맞게 괜찮은것 같습니다. ^^ 쿠팡체험단 이벤트로 상품을 무료로 제공 받아 작성한 구매 후기입니다.,'

In [10]:
okt = Okt()
# test_data = data['review_r'].iloc[1]
tokenized_doc = okt.pos(test_data)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('좋아요', 'Adjective'), ('아이', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('방학', 'Noun'), ('동안', 'Noun'), ('게임', 'Noun'), ('하는데', 'Verb'), ('TV', 'Alpha'), ('로', 'Noun')]
명사 추출 : 아이 방학 동안 게임 로 쿠팡 체험 단 모니터 클릭 클릭 무료 제공 정말 나사 개 끝 뒤 전원 코드 게임기 바로 정말 사이즈 정말 화질 워낙 요즘 고화질 순 거 아주 그냥 식탁 한쪽 사용 하나 건 스피커 조만간 연결 잭 사서 스피커 사용 것 스포츠 게임 이어폰 끼 수 요 저희 컴퓨터 노트북 게 스피커 꼭 그것 프로 거 화질 오른쪽 면 전원 등 조절 게 블루투스 연결 집 스피커 블루투스 이제 스피커 해결 게임 중 쿠팡 체험 단 이벤트 상품 무료 제공 작성 구매 후기 굿굿 무용 듀얼 모니터 인터넷 용 사용 거 사양 건 필요 보조 용 모니터 중 금액 구매 추석 연휴 일요일 로켓 와우 다음 날 새벽 볼 수 모니터 이유 컴맹 컴퓨터 인터넷 사용 무용 보조 모니터 필요 조립 동봉 선 듀얼 모니터 완성 듀얼 모니터 이제 열 이전 한성 인치 무용 좀 감 맛 업그레이드 가격 그대로 오히려 당황 항상 방향 발전 거 추천 성비 재택근무 시 구매 사이즈 생각 편이 사이즈 크게 살걸 살짝 생각 노트북 케이블 선 잘못 화면 것 난리 불량 줄 알 식겁 모니터 고정 부분 생각 느낌 나사 더 조이 것 그냥 무용 돈 거 안 그냥 일반 모니터 맥북 듀얼 모니터 글씨 살짝 흐릿 정도 화질 분 돈 더 게 후회 안 거 나사 안 드라이버 기도 나사 이 드라이버 필요 십자 드라이버 전 다이소 천 원 드라이버 사서 조립 조립 그냥 나사 조이 간단 작업 맛 거 기본 기능 설치 화질 선 명도 안 크게 건 재택근무 자주 편이 고민 그 가격 거 그냥 사무 인터넷 서핑 용 거 가끔 유튜브 영상 볼 때 활용 거 새 상품 흠집 좀 아래쪽 그 부분 좀 실망 뽑기 잘못 요 조립 수평

In [15]:
n_gram_range = (2, 4)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 793
trigram 다섯개만 출력 : ['가격 그냥' '가격 그냥 사무' '가격 그냥 사무 인터넷' '가격 그대로' '가격 그대로 오히려']


In [16]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([test_data])
candidate_embeddings = model.encode(candidates)

In [17]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['동안 게임 쿠팡 체험', '이유 컴맹 컴퓨터 인터넷', '인터넷 서핑 가끔 유튜브', '게임 이어폰 저희 컴퓨터', '이유 컴맹 컴퓨터']


In [18]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [19]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=50)

['간단 작업', '재택근무 자주 편이 고민', '코드 게임기', '듀얼 모니터 인터넷', '방학 동안 게임 쿠팡']

In [95]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [96]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['이유 컴맹 컴퓨터', '연휴 일요일', '영화 두운 부분', '서핑 가끔 유튜브 영상', '재택근무 자주']